# ORKG data download

This notebook downloads all papers from the ORKG API and stores it in a SQLite database.

### JSON helper functions

In [22]:
import json
import os


# Read name.json data in directory
def read_json(filename: str):
    json_file = os.path.join(filename)
    with open(json_file) as f:
        img_annotations = json.load(f)

    return img_annotations


# Pretty print json data to console
def print_json(data: any, tag=""):
    json_str = json.dumps(data, indent=2, sort_keys=True)
    if tag:
        print(tag, json_str)
    else:
        print(json_str)


# Pretty print json data to file
def write_json(filename: str, data: any):
    with open(filename, "w") as file:
        json.dump(data, file, indent=2, sort_keys=True)

### Initialize SQLite database

In [23]:
import sqlite3

base_dir = "data/"
database_path = os.path.join(base_dir, "datalake.db")

# Open a connection to the SQLite database file
db = sqlite3.connect(database_path)

# Create a cursor object to execute SQL statements
cursor = db.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS orkg_statements (id TEXT PRIMARY KEY, data JSON)")

In [24]:
#cursor.execute("DROP TABLE IF EXISTS orkg_statements")
#print(cursor.fetchall())

### Functions to store data in a SQLite database

In [25]:
def entry_exists(id: str):
    cursor.execute("SELECT * FROM orkg_statements WHERE id = ?", (id,))
    rows = cursor.fetchall()
    return len(rows) > 0


def store_data(id: str, data: dict):
    cursor.execute("INSERT INTO orkg_statements (id, data) VALUES (?, ?)", (id, json.dumps(data)))
    db.commit()

### Download all paper ids

The predicate P30 is used to get all paper ids.

In [ ]:
import requests


def get_predicates(predicate: str, page=int(0), size=int(100)):
    url = "http://orkg.org/api/statements/predicate/" + predicate

    params = {
        "page": page,
        "size": size,
        "sort": True
    }

    response = requests.get(url, params=params, timeout=6)
    return response.json()

In [27]:
page = 0
pages = 1

In [28]:
import time

# Download all predicate P30 data, this contains all paper ids
# These data is then used to download the corresponding paper data by subject

paper_ids = set()

while True:
    print(f"Download page {page + 1} of {pages}")

    try:
        data = get_predicates("P30", page=page, size=100)
    except:
        # The ORKG API is not very stable, so we need to retry if it fails
        print(f"Error: page={page} --> retrying in 20 seconds...")
        time.sleep(20)
        continue

    content = data['content']

    for entry in content:
        id = entry["subject"]["id"]
        paper_ids.add(id)

    page += 1
    pages = data['totalPages']
    if "last" not in data or data['last']:
        break

Download page 1 of 1
Download page 2 of 271
Download page 3 of 271
Download page 4 of 271
Download page 5 of 271
Download page 6 of 271
Download page 7 of 271
Download page 8 of 271
Download page 9 of 271
Download page 10 of 271
Download page 11 of 271
Download page 12 of 271
Download page 13 of 271
Download page 14 of 271
Download page 15 of 271
Download page 16 of 271
Download page 17 of 271
Download page 18 of 271
Download page 19 of 271
Download page 20 of 271
Download page 21 of 271
Download page 22 of 271
Download page 23 of 271
Download page 24 of 271
Download page 25 of 271
Download page 26 of 271
Download page 27 of 271
Download page 28 of 271
Download page 29 of 271
Download page 30 of 271
Download page 31 of 271
Download page 32 of 271
Download page 33 of 271
Download page 34 of 271
Download page 35 of 271
Download page 36 of 271
Download page 37 of 271
Download page 38 of 271
Download page 39 of 271
Download page 40 of 271
Download page 41 of 271
Download page 42 of 271
Dow

In [29]:
paper_ids = list(paper_ids)
print("Papers", len(paper_ids))

Papers 26737


### Download paper data

After downloading all paper ids, we can download the corresponding paper data.

In [30]:
def get_statement(paper_id: str):
    url = "https://orkg.org/api/statements/" + paper_id + "/bundle/"
    response = requests.get(url, timeout=8)
    data = response.json()

    return data

In [31]:
from tqdm import tqdm

progress_bar = tqdm(
    leave=False,
    ncols=80,  # Limit the progress bar to a fixed width
    total=len(paper_ids)
)

index = 0

while index < len(paper_ids):
    paper_id = paper_ids[index]
    if entry_exists(paper_id):
        progress_bar.update(1)
        index += 1
        continue

    try:
        data = get_statement(paper_id)
    except:
        # The ORKG API is not very stable, so we need to retry if it fails
        print(f"Error: index={index} paper_id={paper_id} --> retrying in 60 seconds...")
        time.sleep(60)
        continue

    if "error" in data:
        print(f"Error: index={index} paper_id={paper_id}")
        print_json(data)
        break

    store_data(paper_id, data)
    progress_bar.update(1)
    index += 1

progress_bar.close()

 96%|█████████████████████████████████▋ | 25742/26737 [00:26<00:00, 7969.10it/s]

Error: index=25742 paper_id=R164478 --> retrying in 60 seconds...


 96%|███████████████████████████████████▋ | 25746/26737 [01:29<00:29, 34.00it/s]

Error: index=25763 paper_id=R110705 --> retrying in 60 seconds...


 96%|███████████████████████████████████▋ | 25764/26737 [02:40<01:20, 12.14it/s]

...

Error: index=26551 paper_id=R497096 --> retrying in 60 seconds...


 99%|████████████████████████████████████▊| 26558/26737 [39:26<11:56,  4.01s/it]

Error: index=26558 paper_id=R565868 --> retrying in 60 seconds...


100%|████████████████████████████████████▉| 26707/26737 [41:03<00:03,  9.48it/s]

Error: index=26708 paper_id=R574425 --> retrying in 60 seconds...


In [32]:
# Print count of database table orkg
cursor.execute("SELECT COUNT(*) FROM orkg_statements")
print(cursor.fetchone()[0])

26745


In [ ]:
cursor.close()